In [22]:
import pandas as pd
import numpy as np
import math
import copy
import QSTK.qstkutil.qsdateutil as du
import datetime as dt
import QSTK.qstkutil.DataAccess as da
import QSTK.qstkutil.tsutil as tsu
import QSTK.qstkstudy.EventProfiler as ep
import csv
import datetime as dt

#构造
def price_matrix(dt_start, dt_end , ls_symbols):
    dt_timeofday = dt.timedelta(hours=16)
    ldt_timestamps = du.getNYSEdays(dt_start, dt_end, dt_timeofday)
    c_dataobj = da.DataAccess('Yahoo')
    ls_keys = ['close']
    ldf_data = c_dataobj.get_data(ldt_timestamps, ls_symbols, ls_keys)   
    d_data = dict(zip(ls_keys, ldf_data))
    df_close = d_data['close']          
    return df_close

def read_orders(filename='orders.csv' , cash=1000000):
    dates=[]
    symbols=[]
    reader = csv.reader(open(filename , 'rU') , delimiter=',')
    for row in reader:
        dates.append(dt.datetime(int(row[0]), int(row[1]), int(row[2])))
        symbols.append(row[3])
    #remove duplicate
    dates = list(set(dates))
    #多增加一天
    dates.append(dates[len(dates)-1] + dt.timedelta(days=1))
    symbols = list(set(symbols))
    dt_start = dates[0]
    dt_end = dates[len(dates)-1]
    dt_timeofday = dt.timedelta(hours=16)
    #dates为交易日期
    dates = du.getNYSEdays(dt_start, dt_end, dt_timeofday)
    
    
    
    #构建交易矩阵
    trade_matrix = pd.DataFrame(0 , index=dates , columns=symbols)
    trade_matrix['CASH'] = cash
    
    reader = csv.reader(open(filename , 'rU') , delimiter=',')
    for row in reader:
        if(row[4] == 'Sell'):
            trade_matrix[row[3]].ix[dt.datetime(int(row[0]), int(row[1]), int(row[2])) + dt.timedelta(hours=16)] = -1 * int(row[5])
        else:
            trade_matrix[row[3]].ix[dt.datetime(int(row[0]), int(row[1]), int(row[2])) + dt.timedelta(hours=16)] = 1 * int(row[5])
    
    print 'trade_matrix'
    print trade_matrix
    
    #仓位矩阵
    holds_matrix = copy.deepcopy(trade_matrix)
    
    
    for s_sym in symbols:
        for i in range(1, len(dates)):
            holds_matrix[s_sym].ix[dates[i]] = holds_matrix[s_sym].ix[dates[i-1]] + trade_matrix[s_sym].ix[dates[i]]
    
    print 'holds_matrix'
    print holds_matrix
    
    
    #求得前复权的收盘价格矩阵
    df_close = price_matrix(dt_start ,dt_end ,symbols)
    
    
    
    
     
    for s_sym in symbols:
        holds_matrix['CASH'].ix[dates[0]] -= 1 * trade_matrix[s_sym].ix[dates[0]] * df_close[s_sym].ix[dates[0]]
    

    
    for i in range(1, len(dates)):
        last_cash = holds_matrix['CASH'].ix[dates[i-1]]
        trade_sum = 0
        for s_sym in symbols:
            trade_sum += 1 * trade_matrix[s_sym].ix[dates[i]] * df_close[s_sym].ix[dates[i]]
        holds_matrix['CASH'].ix[dates[i]] = last_cash - trade_sum
        
    
    df_close['CASH'] = 1.0
    print 'df_close'
    print df_close
    
    print 'holds_matrix'
    print holds_matrix

    df_val =df_close * holds_matrix
    
    print df_val['AAPL'] + df_val['CASH']
    
    
if __name__ == '__main__':
    read_orders()



trade_matrix
                     AAPL     CASH
2011-01-05 16:00:00  1500  1000000
2011-01-06 16:00:00     0  1000000
2011-01-07 16:00:00     0  1000000
2011-01-10 16:00:00     0  1000000
2011-01-11 16:00:00     0  1000000
2011-01-12 16:00:00     0  1000000
2011-01-13 16:00:00     0  1000000
2011-01-14 16:00:00     0  1000000
2011-01-18 16:00:00     0  1000000
2011-01-19 16:00:00     0  1000000
2011-01-20 16:00:00 -1500  1000000
holds_matrix
                     AAPL     CASH
2011-01-05 16:00:00  1500  1000000
2011-01-06 16:00:00  1500  1000000
2011-01-07 16:00:00  1500  1000000
2011-01-10 16:00:00  1500  1000000
2011-01-11 16:00:00  1500  1000000
2011-01-12 16:00:00  1500  1000000
2011-01-13 16:00:00  1500  1000000
2011-01-14 16:00:00  1500  1000000
2011-01-18 16:00:00  1500  1000000
2011-01-19 16:00:00  1500  1000000
2011-01-20 16:00:00     0  1000000
df_close
                       AAPL  CASH
2011-01-05 16:00:00  332.57     1
2011-01-06 16:00:00  332.30     1
2011-01-07 16:00:00  33